In [1]:
import pandas as pd
from sklearn.model_selection import GroupKFold
import numpy as np
from collections import OrderedDict
from tqdm.auto import trange, tqdm

In [2]:
train_descriptions = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
)
train_coord_df = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv"
)
train_coord_df = train_coord_df.merge(
    train_descriptions, on=["series_id", "study_id"], how="left"
)
train_df = pd.read_csv(
    "rsna-2024-lumbar-spine-degenerative-classification/train.csv"
)
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(
    GroupKFold(n_splits=5).split(
        train_df, np.arange(len(train_df)), train_df.study_id
    )
):
    train_df.loc[valid_index, "fold_id"] = i

In [3]:
level_map = OrderedDict(
    l1_l2="L1/L2",
    l2_l3="L2/L3",
    l3_l4="L3/L4",
    l4_l5="L4/L5",
    l5_s1="L5/S1",
)
condition = [
    "spinal_canal_stenosis",
    "left_neural_foraminal_narrowing",
    "right_neural_foraminal_narrowing",
    "left_subarticular_stenosis",
    "right_subarticular_stenosis",
]
df = []
for i in trange(len(train_df)):
    row = train_df.iloc[i]
    row_list = []
    for key in level_map:
        df += [[row.study_id] + [level_map[key]] + row.loc[[c + "_" + key for c in condition]].to_list() + [row.fold_id]]
train_level_df = pd.DataFrame(df, columns=["study_id", "level"] + condition + ["fold_id"])
train_level_df.to_csv("train_level_df.csv", index=False)

  0%|          | 0/1975 [00:00<?, ?it/s]